In [ ]:
# Dependências

In [1]:
%pip install pyspark
%pip install pandas
%pip install xlrd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Funções de cálculo 

In [2]:
import math

def dms_para_dd(graus, minutos, segundos, direcao):
    # Converter para graus decimais
    decimal = graus + (minutos / 60) + (segundos / 3600)
    
    # Ajustar o sinal para Sul e Oeste
    if direcao in ['S', 'W']:
        decimal = -decimal
    
    return decimal

In [3]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # Raio da Terra em km
    R = 6371.0
    
    # Converter de graus para radianos
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    # Diferença de coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Distância
    distance = R * c
    return distance

In [4]:
from pyspark.sql import functions as F
haversine_udf = F.udf(haversine)

In [ ]:
# Spark session

In [5]:
from pyspark.sql import SparkSession
import pandas as pd
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Analytics e BI") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/08 22:54:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/08 22:54:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/08 22:54:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [7]:
pd_recl1 = pd.read_csv('./dadosconsumidor2022.csv', skiprows=1, sep=';')
reclamacoes_1 = spark.createDataFrame(pd_recl1)

In [8]:
pd_recl2= pd.read_csv('./dadosconsumidor2023.csv', skiprows=1, sep=';')
reclamacoes_2 = spark.createDataFrame(pd_recl2)

In [9]:
# Junta as tabelas em uma só

In [10]:
reclamacoes = reclamacoes_1.union(reclamacoes_2)

In [11]:
aeroportos = spark.createDataFrame(pd.read_excel('./aeroportos.xls', skiprows=2))

In [12]:
empresas_aereas = spark.createDataFrame(pd.read_excel('./empresas_aereas.xls', skiprows=3))\
                        .drop('Unnamed: 0','Unnamed: 3')

In [13]:
voos = spark.read.csv('./VRA_20240916110513.csv', sep =';' ,header=True)

In [39]:
print(f"Voos: {voos}\n")
print(f"empresas_aereas: {empresas_aereas}\n")
print(f"aeroportos: {aeroportos}\n")
print(f"reclamacoes: {reclamacoes}\n")

Voos: DataFrame[Sigla ICAO Empresa Aérea: string, Empresa Aérea: string, Número Voo: string, Código DI: string, Código Tipo Linha: string, Modelo Equipamento: string, Número de Assentos: string, Sigla ICAO Aeroporto Origem: string, Descrição Aeroporto Origem: string, Partida Prevista: string, Partida Real: string, Sigla ICAO Aeroporto Destino: string, Descrição Aeroporto Destino: string, Chegada Prevista: string, Chegada Real: string, Situação Voo: string, Justificativa: string, Referência: string, Situação Partida: string, Situação Chegada: string]

empresas_aereas: DataFrame[Sigla OACI: string, Nome Empresas: string, Nacional ou Estrangeira: string]

aeroportos: DataFrame[CÓDIGO OACI: string, CIAD: string, NOME: string, MUNICÍPIO ATENDIDO: string, UF: string, LATITUDE: string, LONGITUDE: string, ALTITUDE: string, OPERAÇÃO: string, DESIGNAÇÃO: string, COMPRIMENTO: string, LARGURA: string, RESISTÊNCIA: string, SUPERFÍCIE: string, DESIGNAÇÃO.1: string, COMPRIMENTO.1: string, LARGURA.1: 

In [36]:
# Pega apenas voos domésticos

In [40]:
voos_domesticos = voos.filter(
    (F.col('Descrição Aeroporto Origem').contains('BRASIL'))
    &
    (F.col('Descrição Aeroporto Destino').contains('BRASIL'))
)

In [41]:
# Conta voos domésticos

In [42]:
voos_domesticos.count()

1304313

In [43]:
# Tratando a base de aeroportos

In [14]:
aeroportos.printSchema()

root
 |-- CÓDIGO OACI: string (nullable = true)
 |-- CIAD: string (nullable = true)
 |-- NOME: string (nullable = true)
 |-- MUNICÍPIO ATENDIDO: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- ALTITUDE: string (nullable = true)
 |-- OPERAÇÃO: string (nullable = true)
 |-- DESIGNAÇÃO: string (nullable = true)
 |-- COMPRIMENTO: string (nullable = true)
 |-- LARGURA: string (nullable = true)
 |-- RESISTÊNCIA: string (nullable = true)
 |-- SUPERFÍCIE: string (nullable = true)
 |-- DESIGNAÇÃO.1: string (nullable = true)
 |-- COMPRIMENTO.1: string (nullable = true)
 |-- LARGURA.1: string (nullable = true)
 |-- RESISTÊNCIA.1: string (nullable = true)
 |-- SUPERFÍCIE.1: string (nullable = true)
 |-- DESIGNAÇÃO.2: string (nullable = true)
 |-- COMPRIMENTO.2: string (nullable = true)
 |-- LARGURA.2: string (nullable = true)
 |-- RESISTÊNCIA.2: string (nullable = true)
 |-- SUPERFÍCIE.2: string (nulla

In [15]:
aeroportos.withColumn('split_lat', F.split('LATITUDE',' '))\
          .withColumn('split_long', F.split('LONGITUDE', ' '))\
          .show(1, False)

24/10/08 22:55:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+------+---------------------+---------------------+---+-------------+--------------+--------+-------------------------------+----------+-----------+-------+--------------+----------+------------+-------------+---------+-------------+------------+------------+-------------+---------+-------------+------------+-----------------------+------------------------+---------+-------------+------------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+---------------------------------+------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------+----------------+---------------------------+-------------+---+---+-------------+--------------+--------+----+----------+-------------------------+-----------+------------------+-------------------+
|CÓDIGO OACI|CIAD  |NOME                 |MUNICÍPIO ATENDIDO   |UF |LATITUDE     |LONGITUDE     |ALTITUDE|OPE

In [17]:
colunas_df_aeroportos = ['CÓDIGO OACI','LATITUDE', 'LONGITUDE']
aeroportos_origem = aeroportos.select(colunas_df_aeroportos)
aeroportos_dest = aeroportos.select(colunas_df_aeroportos)

for coluna in aeroportos.columns:
  aeroportos_origem = aeroportos_origem.withColumnRenamed(coluna, f'{coluna}_orig')

for coluna in aeroportos.columns:
  aeroportos_dest = aeroportos_dest.withColumnRenamed(coluna, f'{coluna}_dest')

In [18]:
print("Aeroporto dest")
aeroportos_dest.printSchema()
print("Aeroporto Origem")
aeroportos_origem.printSchema()

Aeroporto dest
root
 |-- CÓDIGO OACI_dest: string (nullable = true)
 |-- LATITUDE_dest: string (nullable = true)
 |-- LONGITUDE_dest: string (nullable = true)

Aeroporto Origem
root
 |-- CÓDIGO OACI_orig: string (nullable = true)
 |-- LATITUDE_orig: string (nullable = true)
 |-- LONGITUDE_orig: string (nullable = true)



In [45]:
aeroportos_origem.show(1, False)
aeroportos_dest.show(1, True)

+----------------+-------------+--------------+
|CÓDIGO OACI_orig|LATITUDE_orig|LONGITUDE_orig|
+----------------+-------------+--------------+
|SBAA            |8° 20' 55'' S|49° 18' 11'' W|
+----------------+-------------+--------------+
only showing top 1 row

+----------------+-------------+--------------+
|CÓDIGO OACI_dest|LATITUDE_dest|LONGITUDE_dest|
+----------------+-------------+--------------+
|            SBAA|8° 20' 55'' S|49° 18' 11'' W|
+----------------+-------------+--------------+
only showing top 1 row



In [55]:
aeroportos_voo_orig = voos.join(aeroportos_origem, (F.col('Sigla ICAO Aeroporto Origem') == F.col('CÓDIGO OACI_orig')))

aeroportos_voo_orig_dest = aeroportos_voo_orig.join(aeroportos_dest, (F.col('Sigla ICAO Aeroporto Origem') == F.col('CÓDIGO OACI_dest')), 'left')

In [58]:
aeroportos_voo_orig.show(1, False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | ACN                                                   
 Empresa Aérea                | AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA)           
 Número Voo                   | 5991                                                  
 Código DI                    | 6                                                     
 Código Tipo Linha            | N                                                     
 Modelo Equipamento           | C208                                                  
 Número de Assentos           | 0                                                     
 Sigla ICAO Aeroporto Origem  | SBPS                                                  
 Descrição Aeroporto Origem   | PORTO SEGURO - PORTO SEGURO - BA - BRASIL             
 Partida Prevista             | NULL                                                  
 Partida Real                 | 28/01/2023 

In [59]:
aeroportos.show(1, False, vertical=True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------
 CÓDIGO OACI                                                                                      | SBAA                            
 CIAD                                                                                             | PA0008                          
 NOME                                                                                             | CONCEIÇÃO DO ARAGUAIA           
 MUNICÍPIO ATENDIDO                                                                               | CONCEIÇÃO DO ARAGUAIA           
 UF                                                                                               | PA                              
 LATITUDE                                                                                         | 8° 20' 55'' S                   
 LONGITUDE                                                           

In [60]:
colunas_df_aeroportos = ['CÓDIGO OACI', 'LATITUDE', 'LONGITUDE']
aeroportos_origem_fmt = aeroportos.select(colunas_df_aeroportos).withColumnRenamed('CÓDIGO OACI', 'CÓDIGO OACI_orig')
aeroportos_dest_fmt = aeroportos.select(colunas_df_aeroportos).withColumnRenamed('CÓDIGO OACI', 'CÓDIGO OACI_dest')


In [62]:
aeroportos_origem_fmt.show(1, False, vertical=True)

-RECORD 0--------------------------
 CÓDIGO OACI_orig | SBAA           
 LATITUDE         | 8° 20' 55'' S  
 LONGITUDE        | 49° 18' 11'' W 
only showing top 1 row



In [67]:
aeroportos_voo_orig_join_aoci = voos.join(aeroportos_origem, (F.col('Sigla ICAO Aeroporto Origem') == F.col('CÓDIGO OACI_orig')))
aeroportos_voo_orig_dest_aoci = aeroportos_voo_orig.join(aeroportos_dest, (F.col('Sigla ICAO Aeroporto Destino') == F.col('CÓDIGO OACI_dest')))


In [99]:
aeroportos_voo_orig_join_aoci.show(1, False, vertical=True)
#aeroportos_voo_orig_dest_aoci.show(1, False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | ACN                                                   
 Empresa Aérea                | AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA)           
 Número Voo                   | 5991                                                  
 Código DI                    | 6                                                     
 Código Tipo Linha            | N                                                     
 Modelo Equipamento           | C208                                                  
 Número de Assentos           | 0                                                     
 Sigla ICAO Aeroporto Origem  | SBPS                                                  
 Descrição Aeroporto Origem   | PORTO SEGURO - PORTO SEGURO - BA - BRASIL             
 Partida Prevista             | NULL                                                  
 Partida Real                 | 28/01/2023 

In [70]:
voos_domesticos.printSchema()

root
 |-- Sigla ICAO Empresa Aérea: string (nullable = true)
 |-- Empresa Aérea: string (nullable = true)
 |-- Número Voo: string (nullable = true)
 |-- Código DI: string (nullable = true)
 |-- Código Tipo Linha: string (nullable = true)
 |-- Modelo Equipamento: string (nullable = true)
 |-- Número de Assentos: string (nullable = true)
 |-- Sigla ICAO Aeroporto Origem: string (nullable = true)
 |-- Descrição Aeroporto Origem: string (nullable = true)
 |-- Partida Prevista: string (nullable = true)
 |-- Partida Real: string (nullable = true)
 |-- Sigla ICAO Aeroporto Destino: string (nullable = true)
 |-- Descrição Aeroporto Destino: string (nullable = true)
 |-- Chegada Prevista: string (nullable = true)
 |-- Chegada Real: string (nullable = true)
 |-- Situação Voo: string (nullable = true)
 |-- Justificativa: string (nullable = true)
 |-- Referência: string (nullable = true)
 |-- Situação Partida: string (nullable = true)
 |-- Situação Chegada: string (nullable = true)
 |-- CÓDIGO OAC

In [74]:
voos_domesticos.show(1, False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | ACN                                                   
 Empresa Aérea                | AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA)           
 Número Voo                   | 5991                                                  
 Código DI                    | 6                                                     
 Código Tipo Linha            | N                                                     
 Modelo Equipamento           | C208                                                  
 Número de Assentos           | 0                                                     
 Sigla ICAO Aeroporto Origem  | SBPS                                                  
 Descrição Aeroporto Origem   | PORTO SEGURO - PORTO SEGURO - BA - BRASIL             
 Partida Prevista             | NULL                                                  
 Partida Real                 | 28/01/2023 

In [ ]:
# Join na longitude latitude destino

In [121]:
voos_domesticos_aux = voos_domesticos.drop('LONGITUDE_dest')
voos_domesticos_aux = voos_domesticos_aux.drop('LATITUDE_dest')
voos_domesticos_aux = voos_domesticos_aux.drop('LATITUDE_orig')
voos_domesticos_aux = voos_domesticos_aux.drop('LONGITUDE_orig')


voos_atualizados_dest = voos_domesticos_aux.join(
    aeroportos_voo_orig_dest_aoci.select(
        col("Sigla ICAO Empresa Aérea"),
        col("Número Voo"),
        col("CÓDIGO OACI_dest").alias("CÓDIGO OACI_dest"),
        col("LATITUDE_dest"),
        col("LONGITUDE_dest")
    ),
    (voos_domesticos["Sigla ICAO Empresa Aérea"] == aeroportos_voo_orig_dest_aoci["Sigla ICAO Empresa Aérea"]) &
    (voos_domesticos["Número Voo"] == aeroportos_voo_orig_dest_aoci["Número Voo"]) &
    (voos_domesticos["CÓDIGO OACI_dest"] == aeroportos_voo_orig_dest_aoci["CÓDIGO OACI_dest"]),
    "left"
)

In [107]:
voos_atualizados_dest.show(1, False, vertical=True)

-RECORD 0---------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | ACN                                                 
 Empresa Aérea                | AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA)         
 Número Voo                   | 5976                                                
 Código DI                    | 6                                                   
 Código Tipo Linha            | N                                                   
 Modelo Equipamento           | C208                                                
 Número de Assentos           | 0                                                   
 Sigla ICAO Aeroporto Origem  | SBPS                                                
 Descrição Aeroporto Origem   | PORTO SEGURO - PORTO SEGURO - BA - BRASIL           
 Partida Prevista             | NULL                                                
 Partida Real                 | 23/05/2023 13:10                 

In [123]:
voos_atualizados_orig = voos_domesticos_aux.join(
    aeroportos_voo_orig_join_aoci.select(
        col("Sigla ICAO Empresa Aérea"),
        col("Número Voo"),
        col("CÓDIGO OACI_orig").alias("CÓDIGO OACI_orig"),
        col("LATITUDE_orig"),
        col("LONGITUDE_orig")
    ),
    (voos_domesticos["Sigla ICAO Empresa Aérea"] == aeroportos_voo_orig_dest_aoci["Sigla ICAO Empresa Aérea"]) &
    (voos_domesticos["Número Voo"] == aeroportos_voo_orig_dest_aoci["Número Voo"]) &
    (voos_domesticos["CÓDIGO OACI_orig"] == aeroportos_voo_orig_dest_aoci["CÓDIGO OACI_orig"]),
    "left"
)

In [114]:
voos_atualizados_orig.show(1, False, vertical=True)
voos_atualizados_dest.show(1, False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | ACN                                                   
 Empresa Aérea                | AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA)           
 Número Voo                   | 5991                                                  
 Código DI                    | 6                                                     
 Código Tipo Linha            | N                                                     
 Modelo Equipamento           | C208                                                  
 Número de Assentos           | 0                                                     
 Sigla ICAO Aeroporto Origem  | SBPS                                                  
 Descrição Aeroporto Origem   | PORTO SEGURO - PORTO SEGURO - BA - BRASIL             
 Partida Prevista             | NULL                                                  
 Partida Real                 | 28/01/2023 

-RECORD 0---------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | ACN                                                 
 Empresa Aérea                | AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA)         
 Número Voo                   | 5976                                                
 Código DI                    | 6                                                   
 Código Tipo Linha            | N                                                   
 Modelo Equipamento           | C208                                                
 Número de Assentos           | 0                                                   
 Sigla ICAO Aeroporto Origem  | SBPS                                                
 Descrição Aeroporto Origem   | PORTO SEGURO - PORTO SEGURO - BA - BRASIL           
 Partida Prevista             | NULL                                                
 Partida Real                 | 23/05/2023 13:10                 

In [124]:
voos_atualizados_dest_renamed = voos_atualizados_dest.select(
    col("Sigla ICAO Empresa Aérea").alias("Sigla ICAO Empresa Aérea Dest"),
    col("Empresa Aérea").alias("Empresa Aérea Dest"),
    col("Número Voo"),
    col("Código DI"),
    col("Código Tipo Linha"),
    col("Modelo Equipamento"),
    col("Número de Assentos"),
    col("Sigla ICAO Aeroporto Destino"),
    col("Descrição Aeroporto Destino"),
    col("Chegada Prevista"),
    col("Chegada Real"),
    col("Situação Voo"),
    col("Justificativa"),
    col("Referência"),
    col("Situação Partida"),
    col("Situação Chegada"),
    col("CÓDIGO OACI_dest").alias("CÓDIGO OACI_dest"),
    col("LATITUDE_dest"),
    col("LONGITUDE_dest")
)

AnalysisException: [AMBIGUOUS_REFERENCE] Reference `Sigla ICAO Empresa Aérea` is ambiguous, could be: [`Sigla ICAO Empresa Aérea`, `Sigla ICAO Empresa Aérea`].

In [115]:
voos_combined = voos_atualizados_orig.join(
    voos_atualizados_dest,
    on=["Sigla ICAO Empresa Aérea", "Número Voo"],
    how="left"
)

In [116]:
voos_combined.show(1, False, vertical=True)

-RECORD 0---------------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | ACN                                                       
 Número Voo                   | 5976                                                      
 Empresa Aérea                | AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA)               
 Código DI                    | 6                                                         
 Código Tipo Linha            | N                                                         
 Modelo Equipamento           | C208                                                      
 Número de Assentos           | 0                                                         
 Sigla ICAO Aeroporto Origem  | SBPS                                                      
 Descrição Aeroporto Origem   | PORTO SEGURO - PORTO SEGURO - BA - BRASIL                 
 Partida Prevista             | NULL                                                      

In [118]:
voos_final = voos_combined.select(
    col("Sigla ICAO Empresa Aérea"),
    col("Empresa Aérea"),
    col("Número Voo"),
    col("Código DI"),
    col("Código Tipo Linha"),
    col("Modelo Equipamento"),
    col("Número de Assentos"),
    col("Sigla ICAO Aeroporto Origem"),
    col("Descrição Aeroporto Origem"),
    col("Partida Prevista"),
    col("Partida Real"),
    col("Sigla ICAO Aeroporto Destino"),
    col("Descrição Aeroporto Destino"),
    col("Chegada Prevista"),
    col("Chegada Real"),
    col("Situação Voo"),
    col("Justificativa"),
    col("Referência"),
    col("Situação Partida"),
    col("Situação Chegada"),
    col("CÓDIGO OACI_orig"),
    col("CÓDIGO OACI_dest").alias("CÓDIGO OACI_dest"),
    col("LATITUDE_orig"),
    col("LONGITUDE_orig"),
    col("LATITUDE_dest"),
    col("LONGITUDE_dest")
)

AnalysisException: [AMBIGUOUS_REFERENCE] Reference `Sigla ICAO Empresa Aérea` is ambiguous, could be: [`Sigla ICAO Empresa Aérea`, `Sigla ICAO Empresa Aérea`, `Sigla ICAO Empresa Aérea`].

In [ ]:
voos_final.show(truncate=False)

In [71]:
from pyspark.sql.functions import col
voos_domesticos = voos_final \
    .withColumn('LATITUDE_orig', col('LATITUDE_orig').cast('float')) \
    .withColumn('LONGITUDE_orig', col('LONGITUDE_orig').cast('float')) \
    .withColumn('LATITUDE_dest', col('LATITUDE_dest').cast('float')) \
    .withColumn('LONGITUDE_dest', col('LONGITUDE_dest').cast('float'))

In [ ]:
# Calcula a distância do aeroporto origem para o aeorporto destino

In [39]:
voos_domesticos = voos_domesticos.withColumn('Distância (km)', haversine_udf(
    F.col('LATITUDE_orig'), 
    F.col('LONGITUDE_orig'), 
    F.col('LATITUDE_dest'), 
    F.col('LONGITUDE_dest')
))

In [40]:
df = voos_domesticos.drop('Distância (km)')